In [23]:
from sympy import symbols, Matrix, sqrt, pi, exp, cos, sin, solve
import math

In [24]:
theta, phi = symbols('theta phi')

In [25]:
# quarter-wave plate matrices
QWP_0 = Matrix([[1, 0], [0, exp(pi/2 * 1j)]])  
rotation_matrix = Matrix([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
rotation_inverse_matrix = Matrix([[cos(theta), sin(theta)], [-sin(theta), cos(theta)]])
QWP_theta = rotation_matrix * QWP_0 * rotation_inverse_matrix
QWP_phi = rotation_matrix.subs(theta, phi) * QWP_0 * rotation_inverse_matrix.subs(theta, phi)

# half-wave plate matrices
HWP_0 = Matrix([[1, 0], [0, -1]])  
HWP_theta = rotation_matrix * HWP_0 * rotation_inverse_matrix
HWP_phi = rotation_matrix.subs(theta, phi) * HWP_0 * rotation_inverse_matrix.subs(theta, phi)

In [26]:
QWP_0

Matrix([
[1,             0],
[0, exp(0.5*I*pi)]])

In [27]:
QWP_theta.simplify()
QWP_theta 

Matrix([
[I*sin(theta)**2 + cos(theta)**2,          (1 - I)*sin(2*theta)/2],
[         (1 - I)*sin(2*theta)/2, sin(theta)**2 + I*cos(theta)**2]])

In [28]:
QWP_phi.simplify()
QWP_phi

Matrix([
[I*sin(phi)**2 + cos(phi)**2,        (1 - I)*sin(2*phi)/2],
[       (1 - I)*sin(2*phi)/2, sin(phi)**2 + I*cos(phi)**2]])

In [29]:
HWP_0

Matrix([
[1,  0],
[0, -1]])

In [30]:
HWP_theta.simplify()
HWP_theta

Matrix([
[cos(2*theta),  sin(2*theta)],
[sin(2*theta), -cos(2*theta)]])

In [31]:
HWP_phi.simplify()
HWP_phi

Matrix([
[cos(2*phi),  sin(2*phi)],
[sin(2*phi), -cos(2*phi)]])

In [32]:
HP = Matrix([1, 0])
DP = 1/sqrt(2)*Matrix([1, 1])
LC = 1/sqrt(2)*Matrix([1, 1j])
RC = 1/sqrt(2)*Matrix([1, -1j])
VP = Matrix([0, 1])

In [33]:
f1 = QWP_theta * HP
f1.simplify()

In [34]:
f1

Matrix([
[I*sin(theta)**2 + cos(theta)**2],
[         (1 - I)*sin(2*theta)/2]])

In [35]:
f2 = QWP_phi * f1
f2.simplify()

In [36]:
f2

Matrix([
[(I*sin(phi)**2 + cos(phi)**2)*(I*sin(theta)**2 + cos(theta)**2) + (1 - I)**2*sin(2*phi)*sin(2*theta)/4],
[ (1 - I)*((I*sin(theta)**2 + cos(theta)**2)*sin(2*phi) + (sin(phi)**2 + I*cos(phi)**2)*sin(2*theta))/2]])

In [58]:
f2_new = f2.subs(phi, theta)
f2_new.simplify()
f2_new

Matrix([
[(I*sin(theta)**2 + cos(theta)**2)**2 + (1 - I)**2*sin(2*theta)**2/4],
[                                                       sin(2*theta)]])

In [37]:
f3 = HWP_phi * f1
f3.simplify()

In [38]:
f3

Matrix([
[(I*sin(theta)**2 + cos(theta)**2)*cos(2*phi) + (1 - I)*sin(2*phi)*sin(2*theta)/2],
[                                 (1 - I)*(I*sin(2*phi) + sin(2*phi - 2*theta))/2]])

In [46]:
f3_new = f3.subs(phi, theta/2)
f3_new.simplify()
f3_new

Matrix([
[  cos(theta)],
[I*sin(theta)]])

In [50]:
f4 = HWP_theta * f3_new
f4.simplify()
f4

Matrix([
[(2 - 2*I)*cos(theta)**3 + (-1 + 2*I)*cos(theta)],
[  (-2 + 2*I)*sin(theta)**3 + (2 - I)*sin(theta)]])

In [39]:
from sympy import atan2, re, im, pi, rad, simplify

def complex_phase_difference(vector):
    """
    Calculate the phase difference between two complex components given their matrix and vector representations.

    Args:
    vector (Matrix): A 2x1 column vector representing the complex components.

    Returns:
    float: The phase difference between the two complex components in radians.
    """
    component1 = vector[0]
    component2 = vector[1]

    phase1 = atan2(im(component1), re(component1))
    phase2 = atan2(im(component2), re(component2))
    
    phase_diff = phase2 - phase1
    
    # Handling phase difference to be within the range [-pi, pi)
    phase_diff = phase_diff.rewrite(pi).evalf()
    while phase_diff >= pi:
        phase_diff -= 2*pi
    while phase_diff < -pi:
        phase_diff += 2*pi
        
    phase_diff = simplify(phase_diff)
    
    return phase_diff

In [40]:
v1 = f1.subs(theta, pi/8)
pd_1 = complex_phase_difference(v1)
pd_1

-0.955316618124509

In [41]:
phi_value = pi/16
theta_value = 3*pi/16

v2 = f2.subs([(phi, phi_value), (theta, theta_value)])
pd_2 = complex_phase_difference(v2)
pd_2

0.275533837036719

In [52]:
phi_values = [pi*i/100 for i in range(1,101)]  # Values from 0 to pi
theta_values = [pi*i/100 for i in range(1,101)]  # Values from 0 to pi

for phi_value in phi_values:
    for theta_value in theta_values:
        v2 = f2.subs([(phi, phi_value), (theta, theta_value)])
        pd_2 = complex_phase_difference(v2)
        print("Phi:", math.degrees(phi_value))
        print("Theta:", math.degrees(theta_value))
        print("Phase Difference:", math.degrees(pd_2))
        if abs(pd_2 - pi/2) < 0.00001:  # Tolerance for comparison
            print("Tolerance is reached")
            print("Phi:", math.degrees(phi_value))
            print("Theta:", math.degrees(theta_value))
            print("Phase Difference:", math.degrees(pd_2))

Phi: 1.8
Theta: 1.8
Phase Difference: -3.6780068772428567e-143
Phi: 1.8
Theta: 3.6
Phase Difference: 18.400777570849616
Phi: 1.8
Theta: 5.4
Phase Difference: 26.382205151682847
Phi: 1.8
Theta: 7.2
Phase Difference: 30.51436577700847
Phi: 1.8
Theta: 9.0
Phase Difference: 32.8538118647465
Phi: 1.8
Theta: 10.8
Phase Difference: 34.190666564222056
Phi: 1.8
Theta: 12.6
Phase Difference: 34.883564915772595
Phi: 1.8
Theta: 14.4
Phase Difference: 35.11505525939674
Phi: 1.8
Theta: 16.2
Phase Difference: 34.98532387296466
Phi: 1.8
Theta: 18.0
Phase Difference: 34.55180555270407
Phi: 1.8
Theta: 19.8
Phase Difference: 33.84787298147236
Phi: 1.8
Theta: 21.6
Phase Difference: 32.89249807526656
Phi: 1.8
Theta: 23.400000000000002
Phase Difference: 31.695687610990504
Phi: 1.8
Theta: 25.2
Phase Difference: 30.261833096615764
Phi: 1.8
Theta: 27.0
Phase Difference: 28.59201275666095
Phi: 1.8
Theta: 28.8
Phase Difference: 26.685786590150222
Phi: 1.8
Theta: 30.599999999999998
Phase Difference: 24.5427794988

TypeError: Invalid comparison of non-real -1.5707963267949 + 168.434764876067*I

In [ ]:
phi_value = pi/20
theta_value = pi/10

v3_eval = f3.subs([(phi, phi_value), (theta, theta_value)])
v3_eval.simplify()
v3_eval

Matrix([
[sqrt(2)*(sqrt(2*sqrt(5) + 10)*(2*sqrt(5) + 10 + (2 - 2*I)*sqrt(10 - 2*sqrt(5)) + I*(1 - sqrt(5))**2)/128 - (1 - sqrt(5))*((1 - sqrt(5))**2 + (2 - 2*I)*sqrt(10 - 2*sqrt(5)) + I*(2*sqrt(5) + 10))/128)],
[                                           (1 - I)*(-2*sqrt(10) - sqrt(5 - sqrt(5)) + sqrt(25 - 5*sqrt(5)) + sqrt(5*sqrt(5) + 25) + (1 - 4*I)*sqrt(sqrt(5) + 5) + sqrt(2)*(2 - 2*I) + 2*sqrt(10)*I)/32]])

In [ ]:
pd = complex_phase_difference(v3_eval)
pd

-1.57079632679490

In [ ]:
phi_values = [pi*i/100 for i in range(101)]  # Values from 0 to pi
theta_values = [pi*i/100 for i in range(101)]  # Values from 0 to pi

for phi_value in phi_values:
    for theta_value in theta_values:
        v2 = f2.subs([(phi, phi_value), (theta, theta_value)])
        pd_2 = complex_phase_difference(v2)
        if abs(pd_2 - pi/2) < 0.001:  # Tolerance for comparison
            print("Phi:", phi_value)
            print("Theta:", theta_value)
            print("Phase Difference:", pd_2)

Phi: 0
Theta: pi/4
Phase Difference: 1.57079632679490
Phi: 0
Theta: 3*pi/4
Phase Difference: 1.57079632679490


KeyboardInterrupt: 

In [ ]:
print(math.degrees(0.334))

19.136790357369495
